Very early on I wrote this strategy which I call Flatten.   The idea is very simple, just invert the distribution that the opponent expects you to play and play that instead most of the time (controlled by pFlatten).  The idea is that your play then appears completely random to your opponent so you can sneak in some exploitation.  This is only 68 lines, but very dissapointingly at the end of the contest it's my second highest scoring agent - it currently has score 990.3 and effective rank 77 on 29Jan21 (a repeat submission has just tanked).

In [ ]:
import math
import random
import numpy as np

pFlatten = 0.7 # prob of playing Flatten, else do simple predict
offset   = 2.0
halfLife = 100.0
countPow = math.exp(math.log(2)/halfLife)

naction = 3
reward  = np.array([[0.0, -1.0, 1.0], [1.0, 0.0, -1.0], [-1.0, 1.0, 0.0]])

class Agent:
  def __init__(self):
    self.countInc = 1e-30
    self.countOp  = self.countInc * np.ones((naction, naction, naction))
    self.countAg  = self.countInc * np.ones((naction, naction, naction))
    self.histAgent = []    # Agent history
    self.histOpponent = [] # Opponent history
    self.nwin = 0
    
  # make a move
  def move(self, lastOpponentAction, step):
    if step == 0:
      dist = np.ones(naction)
    else:
      # store last opponent action
      self.histOpponent.append(lastOpponentAction)

      # score last game
      self.nwin += reward[self.histAgent[-1], lastOpponentAction]
      print('step: ', step, 'win: ', self.nwin)

      if step > 1:
        # increment predictors
        self.countOp[self.histOpponent[-2], self.histAgent[-2], self.histOpponent[-1]] += self.countInc
        self.countAg[self.histOpponent[-2], self.histAgent[-2], self.histAgent[-1]] += self.countInc
                    
      # decide on what strategy to play
      if len(self.histOpponent) < 2:
        dist = np.ones(naction)
      else:
        if random.random() < pFlatten:
          # stochastically flatten the distribution
          count = self.countAg[self.histOpponent[-1], self.histAgent[-1]]
          dist  = (offset + 1) * count.max() - offset * count.min() - count
        else:
          # simple prediction of opponent
          count = self.countOp[self.histOpponent[-1], self.histAgent[-1]]
          gain  = np.dot(reward, count)
          dist  = gain + gain.min()
          
    agentAction = random.choices(range(naction), weights=dist)[0]
    self.histAgent.append(agentAction)
    self.countInc *= countPow
    
    return(agentAction)

agent = None

def main(observation, configuration):
  global agent

  if agent == None:
    agent = Agent()
    return agent.move(None, observation.step)
  else:
    return agent.move(observation.lastOpponentAction, observation.step)